In [17]:
from pathlib import Path
import numpy as np
import torch.nn as nn
import torch
from dataclasses import dataclass
from pathlib import Path

import miditok
from miditok import REMI, TokenizerConfig
from miditok.pytorch_data import DataCollator, DatasetMIDI
from torch.utils.data import DataLoader
from miditok import REMI, TokenizerConfig
from midi_player import MIDIPlayer
from model import DatasetMIDI, PopTransformer


## Tokenization Check

In [28]:
test_track = "/Users/stevenkao/workspace/music-hw-3/pop1k7/midi_analyzed/src_001/2.mid" 
MIDIPlayer(test_track, 400)  

In [ ]:
miditok.constants.piano

False

In [29]:
tokenizer = REMI(
    TokenizerConfig(
        use_tempos=True, 
        use_pitchdrum_tokens=False, 
        beat_res={(0, 4): 16, (4, 12): 8},
    )
)

print("vocab size", tokenizer.vocab_size)

z = tokenizer.encode(test_track)
tokens = torch.tensor(z[0].ids)
tokens = tokens.view(1, -1)
score = tokenizer.decode(tokens)
score.dump_midi("sample.mid")
MIDIPlayer("/Users/stevenkao/workspace/music-hw-3/sample.mid", 400)

vocab size 349


In [31]:
for k in tokenizer.vocab.keys():
    print(k)

PAD_None
BOS_None
EOS_None
MASK_None
Bar_None
Pitch_21
Pitch_22
Pitch_23
Pitch_24
Pitch_25
Pitch_26
Pitch_27
Pitch_28
Pitch_29
Pitch_30
Pitch_31
Pitch_32
Pitch_33
Pitch_34
Pitch_35
Pitch_36
Pitch_37
Pitch_38
Pitch_39
Pitch_40
Pitch_41
Pitch_42
Pitch_43
Pitch_44
Pitch_45
Pitch_46
Pitch_47
Pitch_48
Pitch_49
Pitch_50
Pitch_51
Pitch_52
Pitch_53
Pitch_54
Pitch_55
Pitch_56
Pitch_57
Pitch_58
Pitch_59
Pitch_60
Pitch_61
Pitch_62
Pitch_63
Pitch_64
Pitch_65
Pitch_66
Pitch_67
Pitch_68
Pitch_69
Pitch_70
Pitch_71
Pitch_72
Pitch_73
Pitch_74
Pitch_75
Pitch_76
Pitch_77
Pitch_78
Pitch_79
Pitch_80
Pitch_81
Pitch_82
Pitch_83
Pitch_84
Pitch_85
Pitch_86
Pitch_87
Pitch_88
Pitch_89
Pitch_90
Pitch_91
Pitch_92
Pitch_93
Pitch_94
Pitch_95
Pitch_96
Pitch_97
Pitch_98
Pitch_99
Pitch_100
Pitch_101
Pitch_102
Pitch_103
Pitch_104
Pitch_105
Pitch_106
Pitch_107
Pitch_108
Velocity_3
Velocity_7
Velocity_11
Velocity_15
Velocity_19
Velocity_23
Velocity_27
Velocity_31
Velocity_35
Velocity_39
Velocity_43
Velocity_47
Velocity_51

## Trainning

In [4]:
config = ModelConfig()
tokenizer = REMI(TokenizerConfig())
config.vocab_size = tokenizer.vocab_size
midi_dir = Path("pop1k7/midi_analyzed")

dataset = DatasetMIDI(
    files_paths=list(midi_dir.glob("**/*.mid")),
    tokenizer=tokenizer,
    max_seq_len=config.max_seq_length,
    bos_token_id=tokenizer["BOS_None"],
    eos_token_id=tokenizer["EOS_None"],
)

collator = DataCollator(tokenizer.pad_token_id)
dataloader = DataLoader(dataset, batch_size=config.batch_size, collate_fn=collator)

bos_token = tokenizer['BOS_None']
eos_token = tokenizer['EOS_None']
pad_token = tokenizer['PAD_None']

In [14]:
tokenizer.vocab

{'PAD_None': 0,
 'BOS_None': 1,
 'EOS_None': 2,
 'MASK_None': 3,
 'Bar_None': 4,
 'Pitch_21': 5,
 'Pitch_22': 6,
 'Pitch_23': 7,
 'Pitch_24': 8,
 'Pitch_25': 9,
 'Pitch_26': 10,
 'Pitch_27': 11,
 'Pitch_28': 12,
 'Pitch_29': 13,
 'Pitch_30': 14,
 'Pitch_31': 15,
 'Pitch_32': 16,
 'Pitch_33': 17,
 'Pitch_34': 18,
 'Pitch_35': 19,
 'Pitch_36': 20,
 'Pitch_37': 21,
 'Pitch_38': 22,
 'Pitch_39': 23,
 'Pitch_40': 24,
 'Pitch_41': 25,
 'Pitch_42': 26,
 'Pitch_43': 27,
 'Pitch_44': 28,
 'Pitch_45': 29,
 'Pitch_46': 30,
 'Pitch_47': 31,
 'Pitch_48': 32,
 'Pitch_49': 33,
 'Pitch_50': 34,
 'Pitch_51': 35,
 'Pitch_52': 36,
 'Pitch_53': 37,
 'Pitch_54': 38,
 'Pitch_55': 39,
 'Pitch_56': 40,
 'Pitch_57': 41,
 'Pitch_58': 42,
 'Pitch_59': 43,
 'Pitch_60': 44,
 'Pitch_61': 45,
 'Pitch_62': 46,
 'Pitch_63': 47,
 'Pitch_64': 48,
 'Pitch_65': 49,
 'Pitch_66': 50,
 'Pitch_67': 51,
 'Pitch_68': 52,
 'Pitch_69': 53,
 'Pitch_70': 54,
 'Pitch_71': 55,
 'Pitch_72': 56,
 'Pitch_73': 57,
 'Pitch_74': 58,
 'Pitc

In [16]:
[(k, v) for k, v in tokenizer.vocab.items() if "Pitch_" in k]

[('Pitch_21', 5),
 ('Pitch_22', 6),
 ('Pitch_23', 7),
 ('Pitch_24', 8),
 ('Pitch_25', 9),
 ('Pitch_26', 10),
 ('Pitch_27', 11),
 ('Pitch_28', 12),
 ('Pitch_29', 13),
 ('Pitch_30', 14),
 ('Pitch_31', 15),
 ('Pitch_32', 16),
 ('Pitch_33', 17),
 ('Pitch_34', 18),
 ('Pitch_35', 19),
 ('Pitch_36', 20),
 ('Pitch_37', 21),
 ('Pitch_38', 22),
 ('Pitch_39', 23),
 ('Pitch_40', 24),
 ('Pitch_41', 25),
 ('Pitch_42', 26),
 ('Pitch_43', 27),
 ('Pitch_44', 28),
 ('Pitch_45', 29),
 ('Pitch_46', 30),
 ('Pitch_47', 31),
 ('Pitch_48', 32),
 ('Pitch_49', 33),
 ('Pitch_50', 34),
 ('Pitch_51', 35),
 ('Pitch_52', 36),
 ('Pitch_53', 37),
 ('Pitch_54', 38),
 ('Pitch_55', 39),
 ('Pitch_56', 40),
 ('Pitch_57', 41),
 ('Pitch_58', 42),
 ('Pitch_59', 43),
 ('Pitch_60', 44),
 ('Pitch_61', 45),
 ('Pitch_62', 46),
 ('Pitch_63', 47),
 ('Pitch_64', 48),
 ('Pitch_65', 49),
 ('Pitch_66', 50),
 ('Pitch_67', 51),
 ('Pitch_68', 52),
 ('Pitch_69', 53),
 ('Pitch_70', 54),
 ('Pitch_71', 55),
 ('Pitch_72', 56),
 ('Pitch_73', 57)

In [ ]:
def sample(tokenizer, model, max_len=2000):
    sample_size = 1
    bos_token = tokenizer['BOS_None']
    eos_token = tokenizer['EOS_None']
    pad_token = tokenizer['PAD_None']
    tokens = torch.tensor([bos_token]).reshape(sample_size, 1) # (batch_n, seq_len)

    for _ in range(max_len):
        padding_mask = torch.tensor([pad_token] * tokens.shape[1]).reshape(sample_size, -1)
        logits = model(tokens, padding_mask) # (batch_n, seq_len, vocab_size)
        prob = torch.nn.functional.softmax(logits, dim=2)
        next_token = torch.multinomial(prob[:, -1], 1) # (batch_n, 1)
        tokens = torch.cat((tokens, next_token), dim=1)

        if next_token[0, -1] == eos_token:
            break

    return tokens

# tokens = sample(tokenizer, model)
# len(tokens)

In [7]:
model = PopTransformer(
    vocab_size=config.vocab_size,
    d_model=config.d_model,
    nhead=config.n_head,
    num_encoder_layers=config.num_encoder_layers,
    dim_feedforward=config.dim_feedforward,
    max_seq_length=config.max_seq_length
)

optim = torch.optim.Adam(model.parameters())
ctriterion = nn.CrossEntropyLoss(ignore_index=tokenizer["PAD_None"])
data = next(iter(dataloader))


/Users/stevenkao/miniconda3/envs/music-hw3/lib/python3.10/site-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [274]:
for i in range(100):
    model.train()
    optim.zero_grad()

    tokens = data["input_ids"]
    mask = data["attention_mask"]

    logits = model(tokens, mask)
    n_batch, n_seq, vocab_size = logits.shape
    logits = logits.view(n_batch * n_seq, vocab_size)
    tokens = tokens.flatten()
    loss = ctriterion(logits, tokens)

    loss.backward()
    optim.step()

    if i % 20 == 0:
        print(loss.item())

4.30021333694458
1.7786098718643188
0.6422502994537354
0.24322421848773956
0.11996977031230927


In [34]:
tokens = sample(tokenizer, model)
print(tokens.shape)

score = tokenizer.decode(tokens)
score.dump_midi("sample.mid")
MIDIPlayer("/Users/stevenkao/workspace/music-hw-3/sample.mid", 100)  

torch.Size([1, 101])


In [ ]:
data = next(iter(dataloader))
tokens = data['input_ids']

tokens = tokens[0:1, :]
score = tokenizer.decode(tokens)
score.dump_midi("sample.mid")

MIDIPlayer("/Users/stevenkao/workspace/music-hw-3/sample.mid", 400)  

In [23]:
from transformers import GPT2Config, GPT2LMHeadModel

bos_token = tokenizer['BOS_None']
eos_token = tokenizer['EOS_None']
pad_token = tokenizer['PAD_None']

gpt_config = GPT2Config(
    vocab_size=config.vocab_size,
    n_positions=config.max_seq_length,
    n_embd=config.d_model,
    n_layer=config.num_encoder_layers,
    n_head=config.n_head,
    bos_token_id=bos_token,
    eos_token_id=eos_token,
)

gpt2 = GPT2LMHeadModel(gpt_config)

In [ ]:

data = next(iter(dataloader))
tokens = data['input_ids']
attention_mask = data['attention_mask']
tokens.shape

out = gpt2(tokens, attention_mask=attention_mask)
out.logits.shape

torch.Size([32, 511, 282])

In [36]:
optim = torch.optim.Adam(gpt2.parameters())
ctriterion = nn.CrossEntropyLoss(ignore_index=tokenizer["PAD_None"])
data = next(iter(dataloader))

for i in range(100):
    gpt2.train()
    optim.zero_grad()

    tokens = data["input_ids"]
    mask = data["attention_mask"]
    out = gpt2(tokens, attention_mask=mask)
    logits = out.logits

    n_batch, n_seq, vocab_size = logits.shape
    logits = logits.view(n_batch * n_seq, vocab_size)
    tokens = tokens.flatten()
    loss = ctriterion(logits, tokens)

    loss.backward()
    optim.step()

    if i % 20 == 0:
        print(loss.item())

4.6560187339782715
1.5040385723114014
0.3334512412548065


KeyboardInterrupt: 

In [44]:
sample_size = 1
bos_token = tokenizer['BOS_None']
eos_token = tokenizer['EOS_None']
pad_token = tokenizer['PAD_None']
tokens = torch.tensor([bos_token]).reshape(sample_size, 1) # (batch_n, seq_len)

sample = gpt2.generate(tokens, do_sample=True, max_length=100)
sample
    # for _ in range(max_len):
    #     padding_mask = torch.tensor([pad_token] * tokens.shape[1]).reshape(sample_size, -1)
    #     logits = model(tokens, padding_mask) # (batch_n, seq_len, vocab_size)
    #     prob = torch.nn.functional.softmax(logits, dim=2)
    #     next_token = torch.multinomial(prob[:, -1], 1) # (batch_n, 1)
    #     tokens = torch.cat((tokens, next_token), dim=1)

    #     if next_token[0, -1] == eos_token:
    #         break

    # return tokens

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


tensor([[  1,   1,   1,   1,   1,   1, 130, 130, 130, 130, 130, 130, 130,  46,
          46,  46,  46,  46,  46,  46,  46,  46, 160, 160,  43,  43,  43,  43,
          43,  43,  43,  43,  43, 197, 197, 197, 197, 105, 104,  38,  38,  38,
          38,  38,  38, 105, 105, 105, 105, 105, 105, 105, 105, 143, 143, 143,
         143, 143, 143, 143, 145, 145, 145, 145, 145, 145, 145, 145, 145, 145,
         145,  54,  54,  54,  54,  54,  54,  54,  54,  54,  54,  54,  54,  54,
          54,  54,  54,  54,  54,  54,  54,  54,  54,  54,  54, 165, 165, 165,
         165, 165]])

In [1]:

tokenizer.vocab

NameError: name 'tokenizer' is not defined